# Turquoise

In [348]:
import math
import string
import operator as ops

## Range

In [442]:
class Range:
    def __init__(self, start, stop=1, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        self.value = None
        self.result_type = Array
        self.report = ['start', 'stop', 'step']
        
    def evaluate(self):
#         self.value = list(range(self.start, self.stop+1, self.step))
        values = []
        num_steps = round((self.stop-self.start)/self.step)+1
        for i in range(num_steps):
            values.append(self.start+(self.step*i))
        self.value = Array(values)
        return self.value

## Tetration

In [385]:
def tetration(n, m):
    r = n
#     m must be an integer
    m = int(m)
#     Repeatedly raise initial value to the previously computed power (e.g., 2 -> 2^2 -> 2^(2^2) -> ...)
    for i in range(m):
        r = n ** r
    return r
tetration.info = "This function handles Turquoise's tetration functionality, which uses the ^^ operator; it is generally impractical due to extremely rapid increases in the function's output as n and m grow, but is included for completeness."

tetration(1.6, 6)

347.6578448566024

## Expression

In [428]:
class Expression:
    def __init__(self, a=None, b=None, op=None):
        self.a, self.b = a, b
        if type(a) is str:
            self.a = float(a)
        if type(b) is str:
            self.b = float(b)
        self.op = op
        self.op_list = {
            '+': ops.add,
            '*': ops.mul,
            '-': ops.sub,
            '/': ops.truediv,
            '//': ops.floordiv,
            '**': ops.pow,
            '^': ops.pow,
            '^^': tetration,
            '%': ops.mod,
            '!': math.factorial,
            '<': ops.lt,
            '<=': ops.le,
            '>': ops.gt,
            '>=': ops.ge,
            '==': ops.eq,
            '!=': ops.ne
        }
        self.report = ['a', 'op', 'b']
        
    def evaluate(self):
        op_name = self.op_list[self.op]
        self.value = op_name(self.a) if self.op in '!' else op_name(self.a, self.b)
        return self.value
    
    def python(self):
        strings = map(str, [self.a, self.op, self.b])
        return ''.join(strings)
    
e = Expression('8', '6', '+')
e.python()

'8.0+6.0'

## Array

In [416]:
class Array:
    def __init__(self, terms=[]):
        self.data = []
        for t in terms:
            if type(t) is list:
                self.data.append(array(t))
            else:
                if t not in list(',;'):
                    self.data.append(t)
    
    def apply(self, op, b, data=None):
        if not data:
            data = self.data
        result = Array()
        for d in data:
            if type(d) is Array:
                result.append(self.apply(op, b, data=d))
            elif type(d) in [int, float]:
                result.append(op(d, b))
        return result
    
    def append(self, x):
        self.data.append(x)
    
    def __add__(self, b): return self.apply(ops.add, b)
    def __mul__(self, b): return self.apply(ops.mul, b)
    def __sub__(self, b): return self.apply(ops.sub, b)
    def __truediv__(self, b): return self.apply(ops.truediv, b)
    def __floordiv__(self, b): return self.apply(ops.floordiv, b)
    def __pow__(self, b): return self.apply(ops.pow, b)
    def __mod__(self, b): return self.apply(ops.mod, b)
    def __lt__(self, b): return self.apply(ops.lt, b)
    def __le__(self, b): return self.apply(ops.le, b)
    def __gt__(self, b): return self.apply(ops.gt, b)
    def __ge__(self, b): return self.apply(ops.ge, b)
    def __eq__(self, b): return self.apply(ops.eq, b)
    def __ne__(self, b): return self.apply(ops.ne, b)
    
    def __str__(self):
        items = ', '.join(map(str, self.data))
        items = '['+items+']'
        return items

In [353]:
# a = [3,3,3]
# b = [4,4,4]
# a@b

In [370]:
'-6'.isnumeric()
float('-6')

-6.0

## Block

In [434]:
class Block:
    def __init__(self, components=None, parsed=None, parser=None, r=None, block_type=None):
        self.components = components
        self.parser = parser
        self.children = []
        self.dtype = None
        self.block_type = block_type
        self.type = block_type
        self.report = []
        
        self.parsed = None
        if self.components:
            comp_vals = []
            for q in self.components:
                if type(q) in [Token]:
                    q = q.string
                    
                if type(q) in [Block]:
                    if q.like('num'):
#                         q = int(q)
                        q = q.evaluate()
                elif self.numeric(q):
                    q = float(q)
                comp_vals.append(q)
            self.parsed = parser(*r(comp_vals))

    def numeric(self, n):
        return all(c in (string.digits + '.-') for c in n)
            
    def add(self, x):
        self.children.append(x)
    
    def evaluate(self):
        return self.parsed.evaluate()
    
    def print(self, level=0, label='Block'):
        indent = ' '*2
#         print(self.type)
#         print(indent + str(self) + '; ' + type(self.parsed).__name__ + '; ' + str(self.type))
        print(indent*level + label + ': ' + type(self.parsed).__name__ + '; ' + str(self.type))
        for c in self.children:
            c.print(level=level+1)
#         for k, v in vars(self.parser).items():
#             print('{}{}:{}'.format(indent, k, v))
        if self.parsed:
            for k in self.parsed.report:
                v = getattr(self.parsed, k)
                print(indent*(level)+'{}{}:{}'.format(indent, k, str(v)))
        if self.report:
            for k in self.report:
                v = getattr(self, k)
                print(indent*(level)+'{}{}:{}'.format(indent, k, str(v)))
                
    def has_tokens(self):
        return any(type(b) is Token for b in self.children)
    
    def like(self, test):
        return self.type and (self.type == test or self.type.startswith(test))
        
    def __getitem__(self, i):
        return self.children[i]
    
    def __setitem__(self, i, j):
        self.children[i] = j

## Token

In [355]:
class Token(Block):
    def __init__(self, string='', token_type=None):
        super().__init__()
        self.string = string
        self.token_type = token_type
        self.type = self.token_type
        self.report = ['string']
    
    def print(self, **kwargs):
        print(str(self))
    
    def __str__(self, **kwargs):
#         super().print(label='Token', **kwargs)
        return 'Token: '+self.string+': '+self.type

## Program

In [412]:
class Program:
    def __init__(self, source):
        self.source = source
#         Create the root node
        self.tree = Block(block_type='root')
        
#         List of characters and their corresponding type
        self.char_sets = {
            'numeric': string.digits + '.-',
            'op': '!@#$%^&*/-+<>=',
            'syntax': '()[]{},;:=|',
            'letter': string.ascii_lowercase
        }
#         List of syntactical patterns to match to generate the program's structure
#         i.e., the grammar
#         Patterns are listed from highest priority to lowest; a 3-parameter range (a:b:c) will be considered before a 2-parameter one (a:b)
        self.patterns = {
#             'range': ['numeric', ':', 'numeric']
#             3-parameter range expression; start:stop:step
            'range1': [
                lambda x: x[0].like('num') and x[1].string == ':' and x[2].like('num') and x[3].string == ':' and x[4].like('num'),
                lambda x: [x[0], x[2], x[4]],
                Range,
                5
            ],
#             2-parameter range expression; start:stop (default step of 1 is used)
            'range2': [
#                 lambda x: [x[0].like('num') and x[1] == ':' and x[2].like('num')],
#                 lambda x: x[0].like('num') and x[1] == ':' and x[2].like('num'),
                lambda x: x[0].like('num') and x[1].string == ':' and x[2].like('num'),
                lambda x: [x[0], x[2]],
                Range,
                3
            ],
#             Factorial expression; 'number!'
            'factorial': [
                lambda x: x[0].like('num') and x[1].string == '!',
                lambda x: [x[0], None, x[1]],
                Expression,
                2
            ],
#             A mathematical expression using infix notation; 'a?b' or '3*5'
            'expression': [
                lambda x: x[0].like('num') and x[1].like('op') and x[2].like('num'),
                lambda x: [x[0], x[2], x[1]],
                Expression,
                3
            ]
        }
        
#         Run the lexer and parser
        self.lex()
        self.parse(self.tree)
    
    def match_types(x, y):
        return all(xi.like(y[i]) for i, xi in enumerate(x))
    
    def char_type(self, x):
#         return list(filter(lambda x: k for k, v in self.char_sets if x in v))[0]
        return [k for k, v in self.char_sets.items() if x in v][0]
    
    def lex(self):
#         Both newlines and pipes, |, separate statements
        statements = self.source.replace('|', '\n').split('\n')
#         Remove empty lines
        statements = list(filter(None, statements))
#         Loop through lines in program
        for s in statements:
            statement_parse = Block(block_type='section')
            token = Token(s[0], token_type=self.char_type(s[0]))
            for c in s[1:]:
                c_type = self.char_type(c)
#                 If character matches the type of the current token, append it
                if c_type == token.token_type:
                    token.string += c
#                 Otherwise, store the token and start a new one
                else:
                    statement_parse.add(token)
                    token = Token(c, token_type=c_type)
            statement_parse.add(token)
            self.tree.add(statement_parse)
    
    def parse(self, block, level=0):
        for j, b in enumerate(block.children):
            self.parse(b)
        i = 0
#         Loop through stored patterns
        for k, v in self.patterns.items():
#             Separate pattern list into the matching rule, resulting block, type of new block to be constructed, and number of terms in the expression
            pattern, result, block_type, num = v
#             num = 3
            section = block[i:i+num]
#             r = lambda x: x[0].like('num') and x[1] == ':' and x[2].like('num')
#             Check that section has at least enough terms to evaluate the rule (e.g., '3 + 4' has 3 terms)
            if len(section) >= num:
#                 print(list(map(str, section)), pattern(section), section[1].like('num'))
                if pattern(section):
#                     print(True, list(map(str, block[i:i+num])))
#                     block[i:i+num] = Block(section, parser=block_type, r=result, block_type='numeric')
#                     Replace sub-blocks or tokens with a single block
                    block[i:i+num] = [Block(section, parser=block_type, r=result, block_type='numeric')]
#                     Update the block in case any more rules are applicable
                    self.parse(block)
                    break
#                 use has_tokens?
                    
    def execute(self, node=None):
#         Default node is the root node
        if not node:
            node = self.tree
        
#         Loop through nodes in tree
        for b in node:
            if b.parser in [Range, Expression]:
                b.parsed.evaluate()
                print(b.parsed.value)
#             Recursively execute subnodes
            else:
#                 for v in b:
#                     self.execute()
               self.execute(node=b)
                    
    def print_tree(self):
        print('\n')
        self.tree.print()
    

## Testing

In [446]:
tests = """
7
1:10%2==1
6:16:.5
8==7
5!=3
5!
8>5
9<=3
51%13
25%3
3**3**3
3^3^3
3:10**2
5:30
1:15**3
1:10*9
3:30:3
20:0:-1
1+2+3
8*9+2
5+10
8*3
20/4
932//7
5+8.3
2**8
2^8
2^^3
"""

M = Program(tests)
M.execute()
# vars(M.tree[0].parsed)
M.print_tree()
# M.tree[0].components[1].like('op')
# M.tree[-1].parser#.components[1].token_type
# store statement and result for each line (?)

[True, False, True, False, True, False, True, False, True, False]
[6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0]
False
True
120
True
False
12.0
1.0
19683.0
19683.0
[9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0, 100.0]
[5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]
[1.0, 8.0, 27.0, 64.0, 125.0, 216.0, 343.0, 512.0, 729.0, 1000.0, 1331.0, 1728.0, 2197.0, 2744.0, 3375.0]
[9.0, 18.0, 27.0, 36.0, 45.0, 54.0, 63.0, 72.0, 81.0, 90.0]
[3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0, 30.0]
[20.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.0]
6.0
74.0
15.0
24.0
5.0
133.0
13.3
256.0
256.0
65536.0


Block: NoneType; root
  Block: NoneType; section
Token: 7: numeric
  Block: NoneType; section
    Block: Expression; numeric
      a:[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.

<ipython-input-428-6c3be09f5b13>:31: DeprecationWarning: Using factorial() with floats is deprecated
  self.value = op_name(self.a) if self.op in '!' else op_name(self.a, self.b)


In [409]:
list(range(20, 0, -1))

[20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [358]:
# M.__dict__
M.tree[0][1].type

IndexError: list index out of range

In [ ]:
bool([False])
f = [3, 7, 2, 1, 8]
f[0:3]